In [1]:
import sys
sys.path.append( '../..' )
import os
import torch
import torch.nn as nn 
import torch.nn.functional as F
from pytorch3d.io import load_obj, save_obj, load_ply
from pytorch3d.structures import Meshes
from pytorch3d.utils import ico_sphere
from pytorch3d.ops import sample_points_from_meshes
from pytorch3d.loss import (
    chamfer_distance, 
    mesh_edge_loss, 
    mesh_laplacian_smoothing, 
    mesh_normal_consistency,
)
import numpy as np
from energy import tangent_kernel

# Set the device
if torch.cuda.is_available():
    device = torch.device("cuda:0")
else:
    device = torch.device("cpu")
    print("WARNING: CPU only, this will be slow!")
import input_output 

In [2]:
experiment_name = 'test'
#experiment_name = 'Bunny'


In [ ]:
data1 = np.loadtxt('/home/juheonlee/learning/neural_varifold/data/processed/airplane/airplane00.txt',delimiter=' ')
data2 = np.loadtxt('/home/juheonlee/learning/neural_varifold/data/processed/airplane/airplane00.txt',delimiter=' ')

In [5]:
class testnet(nn.Module):
    def __init__(self):
        super(testnet, self).__init__()
        self.net = nn.Sequential(nn.Linear(3,64),nn.ReLU(),nn.Linear(64,128),nn.ReLU(),nn.Linear(128,3))
    def forward(self,x):
        return self.net(x)

In [6]:
# optimizer setting
#deform_verts = torch.full(src_mesh.verts_packed().shape, 0.0, device=device, requires_grad=True)
#optimizer = torch.optim.SGD([deform_verts], lr=.01, momentum=0.9)
#optimizer = torch.optim.Adam([deform_verts], lr=.01)
models = testnet().cuda()
optimizer = torch.optim.Adam(models.parameters(), lr=.01)
scheduler = torch.optim.lr_scheduler.MultiStepLR(optimizer, milestones=[501,1001], gamma=0.1)

# Number of optimization steps
Niter = 2001

# loss parameters

# weight for varifold loss
w_varifold = 10.0 
# Weight for the chamfer loss
w_chamfer = 1.0 
# Weight for mesh edge loss
w_edge = 1.0 
# Weight for mesh normal consistency
w_normal = 0.1 
# Weight for mesh laplacian smoothing
w_laplacian = 0.1 
# Plot period for the losses
plot_period = 250


chamfer_losses = []
laplacian_losses = []
edge_losses = []
normal_losses = []

varifold = tangent_kernel(1,1.,0.05,3,mode='NTK1')
def compute_engine(V1,V2,L1,L2,K):
    cst_tmp = []
    n_batch = 10000#4096
    for i in range(len(V1)//n_batch + 1):
        tmp = V1[i*n_batch:(i+1)*n_batch,:]
        l_tmp = L1[i*n_batch:(i+1)*n_batch,:]
        v = torch.matmul(K(tmp,V2),L2)*l_tmp
        cst_tmp.append(v)
    cst = torch.sum(torch.cat(cst_tmp,0))
    return cst

def CompCLNn(F, V):
    if F.shape[1] == 2:
        V0, V1 = V.index_select(0, F[:, 0]), V.index_select(0, F[:, 1])
        C, N  =  (V0 + V1)/2, V1 - V0
    else:
        V0, V1, V2 = V.index_select(0, F[:, 0]), V.index_select(0, F[:, 1]), V.index_select(0, F[:, 2])
        C, N =  (V0 + V1 + V2)/3, .5 * torch.cross(V1 - V0, V2 - V0)

    L = (N ** 2).sum(dim=1)[:, None].sqrt()
    return C, L, N / L, 1#Fun_F

c,l,n,_ = CompCLNn(faces_idx1,verts1)

In [7]:

for i in range(Niter):
    # Initialize optimizer
    optimizer.zero_grad()
    deform_verts = models(sv.cuda()) 
    
    # Deform the mesh
    
    l1 = 1
    l2 = 1
    
    v11 = compute_engine(c1,c1,l1,l1,varifold)
    v22 = compute_engine(c2,c2,l2,l2,varifold) 
    v12 = compute_engine(c2,c1,l2,l1,varifold)

    loss_varifold = v11 + v22 -2*v12

    loss_chamfer, _ = chamfer_distance(c1.unsqueeze(0), c2.unsqueeze(0))

    # and (b) the edge length of the predicted mesh
    loss_edge = mesh_edge_loss(new_src_mesh)
    
    # mesh normal consistency
    loss_normal = mesh_normal_consistency(new_src_mesh)

    # mesh laplacian smoothing
    loss_laplacian = mesh_laplacian_smoothing(new_src_mesh, method="uniform")

    # Weighted sum of the losses
    # note PointNet-NTK1 param = 200 & learning rate 0.01 PointNet-NTK2 = 10 learning rate 0.01
    loss = w_varifold *loss_varifold #+ loss_normal * w_normal # + loss_edge * w_edge + loss_laplacian * w_laplacian 
    # Print the losses
    if i % plot_period==0:
        print('%d Iter: total_loss %.6f Chamfer_loss %.6f Varifold loss %.8f'% (i,loss,loss_chamfer, loss_varifold))
    
    # Save the losses for plotting
    #chamfer_losses.append(float(loss_chamfer.detach().cpu()))
    #edge_losses.append(float(loss_edge.detach().cpu()))
    #normal_losses.append(float(loss_normal.detach().cpu()))
    #laplacian_losses.append(float(loss_laplacian.detach().cpu()))
        
    # Optimization step
    loss.backward()
    optimizer.step()

0 Iter: total_loss 0.061827 Chamfer_loss 0.111240 Varifold loss 0.00618269
250 Iter: total_loss 0.000421 Chamfer_loss 0.036846 Varifold loss 0.00004215
500 Iter: total_loss 0.000526 Chamfer_loss 0.035405 Varifold loss 0.00005256
750 Iter: total_loss 0.000225 Chamfer_loss 0.034568 Varifold loss 0.00002253
1000 Iter: total_loss 0.000193 Chamfer_loss 0.033725 Varifold loss 0.00001926
1250 Iter: total_loss 0.000179 Chamfer_loss 0.033432 Varifold loss 0.00001791
1500 Iter: total_loss 0.000173 Chamfer_loss 0.033120 Varifold loss 0.00001734
1750 Iter: total_loss 0.000317 Chamfer_loss 0.033116 Varifold loss 0.00003167
2000 Iter: total_loss 0.000142 Chamfer_loss 0.032603 Varifold loss 0.00001422


In [8]:
# Fetch the verts and faces of the final predicted mesh
final_verts, final_faces = new_src_mesh.get_mesh_verts_faces(0)

# Scale normalize back to the original target size
final_verts = (final_verts) * scale2 + center2

# Store the predicted mesh using save_obj
#save_obj('./chamfer.obj', final_verts, final_faces)
#save_obj('./charon_trouve.obj', final_verts, final_faces)
save_obj('../../results/pointnet_ntk1_%s.obj'%(experiment_name), final_verts, final_faces)
#save_obj('./pointnet_ntk2.obj', final_verts, final_faces)
print('Done!')

Done!
